In [45]:
import numpy as np
from typing import Callable, Optional, List

# Project Idea: Find the Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## TODOs:
* Something to do $\det(A - \lambda I) = 0$ for eigenvalue
* Something to do $(A - \lambda I)\vec{v} = O$ where $O$ is a zero matrix for eigenvector

## Part 1: Fancy Bisection and Roots Finder

This bisection is to help out when we need to find the root(s) of stuff

In [46]:
"""
Fancier version of bisection

Params:
* f: the function such that we want to find the point that f(x) = 0
* l: initial left bound
* r: initial right bound

Returns: 
* a root in given bound rounded to 6 digits
* None if there's no root or takes >50 iterations
"""
def bisect(f:Callable[[float], float], l:float, r:float) -> Optional[float]:
	if f(l)*f(r) > 0: # bad initial bound
		return None

	g = (l+r)/2
	c = 0
	while abs(f(g)) > 1e-6 and c < 50:
		computed_g = f(g)
		if computed_g * f(l) < 0:
			r = g
		else:
			l = g
		g = (l+r)/2
		c += 1
	
	if abs(f(g)) > 1e-6: # took too long
		return None
	else:
		return round(g, 6)

# bisect(lambda x : np.cos(x) - x, 0, 1)

In [47]:
"""
For finding multiple roots within the given bound

Params:
* f: the function we want to find roots for f(x) = 0
* l: left bound
* r: right bound
* n: how many intervals do we want

Returns: 
* list of roots rounded to 6 digits if any
"""
def find_roots(f:Callable[[float], float], l:float, r:float, n:int) -> List[float]:
	d = (r-l)/n
	roots = []
	for i in range(n):
		cl = l + d*i
		cr = l + d*(i+1)
		res = bisect(f, cl, cr)
		if res != None:
			roots.append(res)
	return roots

# find_roots(lambda x: x**2+x-5, -5, 5, 20)

## Part 2: Finding the Determinant of a Matrix

Determinant is very useful when we have to find the eigenvalues, *and I guess I need to write it from scratch too.*

In [48]:
def det(A:np.ndarray) -> float:
	size, _ = A.shape
	if size == 2:
		return A[0, 0]*A[1, 1] - A[1, 0]*A[0, 1]

	else:
		accumulator = 0
		sign = -1
		for i in range(size):
			sub_matrix = np.array([ [ A[row, col] for col in range(size) if col != i ] for row in range(1, size) ])
			accumulator += (sign**i) * A[0, i]*det(sub_matrix)
		return accumulator

np.random.seed(123)
test_matrix = np.random.randint(1, 10, size=(2, 2))
print("np det:", np.linalg.det(test_matrix))
print("my det:", det(test_matrix))

test_matrix = np.random.randint(1, 20, size=(3, 3))
print(test_matrix)
print("np det:", np.linalg.det(test_matrix))
print("my det:", det(test_matrix))

np det: -15.0
my det: -15
[[11  2  1]
 [18 16 10]
 [ 1 15  1]]
np det: -1235.9999999999995
my det: -1236


In [49]:
def find_eigenvalues(A:np.ndarray, lmbda:float):
	...
